**FINAL VERSION**

This module allows for the prediction of MI given an ECG Diagnosis, outputing a report that contains the new diagnosis according to our ML model.

**Input:**

A 12SL Diagnosis and feature set for a given patient, 3 ML Models (.txt)

**Output:**

Markdown File with ECG Visual, SHAP Plot, 12SL Diagnosis, ML Diagnosis, and a sample decision tree.

In [ ]:
# IMPORT NECESSARY LIGHTGBM & PD Packages
import lightgbm as lgb
import pandas as pd
import shap
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
# GET MODEL TO WORK IN COLAB W/ GPU (GO TO EDIT -> NOTEBOOK SETTINGS -> GPU)
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd

In [ ]:
from re import X
# IMPORT MODELS & Patients
filepath_pos = "/content/model_pos_hyperopt.txt"
filepath_neg = "/content/model_neg_hyperopt.txt"
filepath_localizer = "/content/localization_model.txt"
filepath_patients = "/content/patients.csv"
X_train_neg = pd.read_csv("/content/X_train_neg.csv")
filepath_scaler = "/content/localization_scaler.pkl"
filepath_pca = "/content/localization_pca.pkl"

In [ ]:
# Severity Model
import joblib
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

def predict_threat(input_df, scaler_path, pca_path, model_path):
    """
    Standardizes, applies PCA, and predicts Threat for the given input data.

    Parameters:
        input_df (pd.DataFrame): Raw input data (should match training columns)
        scaler_path (str): Path to saved StandardScaler .pkl file
        pca_path (str): Path to saved PCA .pkl file
        model_path (str): Path to saved LightGBM .txt model

    Returns:
        np.ndarray: Predicted Threat labels
    """
    # Load trained scaler, PCA, and model
    scaler = joblib.load(scaler_path)
    pca = joblib.load(pca_path)
    model = lgb.Booster(model_file=model_path)

    # Drop/ignore any unwanted columns safely
    input_df = input_df.drop(columns=[
        "PatientID", "12SL_Codes", "Phys_Codes", "TestID", "Source",
        "Gender", "PatientAge", "AcquisitionDateTime_DT", "MI_Phys",
        "POffset", "PAxis", "POnset", "Prediction", "PredictionType",
        "MI_Predicted_Final", "MI_Predicted_Again", "Threat"
    ], errors="ignore")

    input_df = input_df.loc[:, ~input_df.columns.str.contains('^P')]
    input_df = input_df.loc[:, ~input_df.columns.str.contains('^PP')]
    input_df = input_df.loc[:, ~input_df.columns.str.contains('PFull')]
    input_df = input_df.loc[:, ~input_df.columns.str.contains('Rate')]
    input_df = input_df.fillna(0)

    # Apply scaler
    X_scaled = scaler.transform(input_df)

    # Apply PCA
    X_pca = pca.transform(X_scaled)

    # Predict
    preds = model.predict(X_pca)
    preds_binary = np.where(preds >= 0.5, 1, 0)  # assuming binary classification
    print(preds)
    return preds_binary

In [ ]:
# ENSEMBLE METHOD
def Model(df, ecg_visual, output_dir="reports"):
    os.makedirs(output_dir, exist_ok=True)

    # --- Feature Cleanup ---
    X_patient = df.drop(columns=[
        "PatientID", "12SL_Codes", "Phys_Codes", "TestID", "Source",
        "Gender", "PatientAge", "AcquisitionDateTime_DT", "MI_Phys",
        "POffset", "PAxis", "POnset", "MI_12SL"
    ])
    X_patient = X_patient.loc[:, ~X_patient.columns.str.contains('^P_')]
    X_patient = X_patient.loc[:, ~X_patient.columns.str.contains('^PP_')]
    X_patient = X_patient.loc[:, ~X_patient.columns.str.contains('PFull')]
    X_patient = X_patient.loc[:, ~X_patient.columns.str.contains('Rate')]
    X_patient = X_patient.select_dtypes(include=['int', 'float', 'bool'])

    # LOAD MODELS
    pos = lgb.Booster(model_file=filepath_pos)
    neg = lgb.Booster(model_file=filepath_neg)
    neg2 = lgb.Booster(model_file=filepath_neg)
    #severity = lgb.Booster(model_file=filepath_localizer)

    # CHECK 12SL Diagnosis
    if df['MI_12SL'].values[0] == 1:
        model_to_use = pos
    else:
        model_to_use = neg

    X_patient_model = X_patient[model_to_use.feature_name()]
    prediction = model_to_use.predict(X_patient_model)

    # SECOND PASS
    if prediction[0] < 0.5:
        X_patient_model = X_patient[neg2.feature_name()]
        prediction = neg2.predict(X_patient_model)
        if prediction[0] < 0.5:
            final_diagnosis = 'NO MI'
        else:
            # Proceed to severity model
            #X_localizer = X_patient.copy()
            # WARNING: NEED SCALER/PCA HERE LATER if needed
            #X_patient_model = X_localizer[severity.feature_name()]
            #prediction = severity.predict(X_patient_model)
            #final_diagnosis = 'NON-ACUTE MI' if prediction[0] > 0.5 else 'ACUTE MI'
            final_diagnosis = 'NON-ACUTE MI' if predict_threat(df,filepath_scaler,filepath_pca, filepath_localizer) == 0 else 'ACUTE MI'
            #final_diagnosis = 'MI'
    else:
        #X_localizer = X_patient.copy()
        # WARNING: NEED SCALER/PCA HERE LATER if needed
        #X_patient_model = X_localizer[severity.feature_name()]
        #prediction = severity.predict(X_patient_model)
        #final_diagnosis = 'NON-ACUTE MI' if prediction[0] > 0.5 else 'ACUTE MI'
        final_diagnosis = 'NON-ACUTE MI' if predict_threat(df,filepath_scaler,filepath_pca, filepath_localizer) == 0 else 'ACUTE MI'
        #final_diagnosis = 'MI'

    # LOCALIZER
    # X_localizer = X_patient.copy()
    # X_localizer_scaled = scaler.transform(X_localizer)

    # X_patient_model = X_patient[severity.feature_name()]
    # prediction = severity.predict(X_patient_model)
    # if (prediction == 0):
    #  prediction = 'NON-ACUTE MI'
    # else:
    #   prediction = 'ACUTE MI'
    # return prediction
    # Prepare fields safely
    patient_id = df['PatientID'].values[0]
    age = df['PatientAge'].values[0] if 'PatientAge' in df and pd.notna(df['PatientAge'].values[0]) else 'Unknown'
    date = patient_record['AcquisitionDateTime_DT'].values[0]
    gender = df['Gender'].values[0] if 'Gender' in df and pd.notna(df['Gender'].values[0]) else 'Unknown'
    sl_mi_value = df['MI_12SL'].values[0] if 'MI_12SL' in df else 'Unknown'
    agreement = (final_diagnosis == sl_mi_value)

    if pd.isna(sl_mi_value):
        sl_mi = 'Unknown'
    else:
        sl_mi = 'MI' if sl_mi_value == 1 else 'No MI'

    physician_mi_value = df['MI_Phys'].values[0] if 'MI_Phys' in df else 'Unknown'
    if pd.isna(physician_mi_value):
        physician_mi = 'Unknown'
    else:
        physician_mi = 'MI' if physician_mi_value == 1 else 'No MI'
    # --- Save SHAP plot ---
    background = X_train_neg.sample(100, random_state = 42)
    explainer = shap.TreeExplainer(model_to_use, model_output = 'probability', feature_perturbation='interventional', data = background)
    shap_values = explainer.shap_values(X_patient_model)

    shap_filename = os.path.join(output_dir, f"{patient_id}_shap.png")

    shap.plots._waterfall.waterfall_legacy(
        explainer.expected_value,
        shap_values[0],
        feature_names=X_patient_model.columns,
        max_display=20,
        show=False
    )
    plt.savefig(shap_filename, bbox_inches='tight')
    plt.close()

   # --- Save ECG plot ---
    ecg_filename = os.path.join(output_dir, f"{patient_id}_ecg.png")

    # Copy the existing ECG PNG into the report folder
    shutil.copy(ecg_visual, ecg_filename)

    # --- Generate Markdown Report ---
    markdown_filename = os.path.join(output_dir, f"{patient_id}_report.md")

    # Save markdown
    with open(markdown_filename, "w") as f:
        f.write(f"# Prediction Report for Patient {patient_id}\n\n")

        f.write("## Patient Metadata\n")
        f.write(f"- **Patient ID:** `{patient_id}`\n")
        f.write(f"- **Age:** `{age}`\n")
        f.write(f"- **Gender:** `{gender}`\n")
        f.write(f"- **Acquisition Date:** `{date}`\n")

        f.write("## Prediction Summary\n")
        f.write(f"- **Ground Truth (MI_Phys):** `{physician_mi}`\n")
        f.write(f"- **12SL Prediction:** `{sl_mi}`\n")
        f.write(f"- **Final ML Model Diagnosis:** `{final_diagnosis}`\n")
        f.write(f"- **Model Probability of MI:** `{prediction[0]:.4f}`\n")
        f.write(f"- **Model and 12SL Agree?:** `{agreement}`\n\n")

        f.write(f"## ECG Visual\n")
        f.write(f"![ECG Plot]({patient_id}_ecg.png)\n\n")

        f.write(f"## SHAP Feature Explanation\n")
        f.write(f"![SHAP Plot]({patient_id}_shap.png)\n\n")

    print(f"Markdown report saved to: {markdown_filename}")

    return final_diagnosis

In [ ]:
# MAIN
import numpy as np  # fake ecg

# Load patients CSV
df_patients = pd.read_csv(filepath_patients)

# Pick a patient manually or by input
patient_id = input("Enter Patient ID (e.g., HR00274): ")

# Find the patient
patient_record = df_patients[df_patients['PatientID'] == patient_id]

if patient_record.empty:
    print(f"Patient {patient_id} not found in dataset.")
else:
    # Example ECG visual
    ecg_visual = 'HR00008_ecg.png'

    # Run the model
    diagnosis = Model(patient_record, ecg_visual)

    print(f"Completed. Final model diagnosis for {patient_id}: {diagnosis}")


Enter Patient ID (e.g., HR00274): HR12857
[LightGBM] [Warning] Ignoring unrecognized parameter 'bagging_by_query' found in model string.
[LightGBM] [Warning] Ignoring unrecognized parameter 'bagging_by_query' found in model string.
[LightGBM] [Warning] Ignoring unrecognized parameter 'bagging_by_query' found in model string.
[0.18069075]
Markdown report saved to: reports/HR12857_report.md
Completed. Final model diagnosis for HR12857: NON-ACUTE MI
